<a href="https://colab.research.google.com/github/Panperception/QKD/blob/main/QKD_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## AI-Assisted QKD: Enhancing Security & Efficiency – Python Simulation Guide
This project will focus on integrating Machine Learning (ML) with Quantum Key Distribution (QKD) to improve security, efficiency, and error correction. The implementation will involve:

Simulating a QKD system (e.g., BB84 protocol).
Generating a quantum channel with noise to model realistic conditions.
Applying AI/ML models to enhance security, key reconciliation, and error correction.

### Initialization:

In [1]:
# %pip uninstall qiskit
!pip install pyqmc
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit-algorithms
!pip install qiskit-nature
!pip install qutip
!pip install ase
!pip install scipy
!nvcc --version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 

In [2]:
# import numpy as np
# from qiskit import QuantumCircuit, execute
# import qiskit_aer as Aer
from qiskit.visualization import array_to_latex, plot_bloch_vector, plot_bloch_multivector, plot_state_qsphere, plot_state_city
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import AerSimulator, Aer

from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature import settings
from qiskit.primitives import Sampler, StatevectorSampler
from qiskit_algorithms import HamiltonianPhaseEstimation, PhaseEstimation
from qiskit.primitives import Estimator
from qiskit_algorithms.optimizers import SLSQP, SPSA, QNSPSA
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit.circuit.library import TwoLocal
from functools import partial as apply_variation
from qiskit.circuit.library import Initialize
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.properties import ParticleNumber
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer, FreezeCoreTransformer
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.operators import ElectronicIntegrals
from pyscf import gto, scf
from ase import Atoms
from ase.build import molecule
from ase.visualize import view
from qiskit_nature.second_q.drivers import PySCFDriver

import numpy as np
import matplotlib.pyplot as plt
import qiskit.quantum_info as qi
import os.path
#import pyqmc.api as pyq
import pyqmc.api as pyq
import h5py
import cmath
import math
import scipy.stats as stats
import qutip
import time, datetime
import pandas as pd


## 1. Simulation Setup in Python
We will use the following libraries:
1. Qiskit for quantum state preparation and measurement.
2. NumPy & SciPy for noise modeling and data processing.
3. TensorFlow/PyTorch for machine learning models.
4. Gym (optional) for reinforcement learning (RL).


### Step 1: Simulating QKD (BB84 Protocol)
BB84 is a widely used QKD protocol where Alice sends qubits in random bases, and Bob measures them in a matching or mismatching basis.

Basic BB84 Implementation:

In [6]:
def generate_bb84_bits(n):
    """Generate random bit string and random bases for Alice"""
    bits = np.random.randint(0, 2, n)
    bases = np.random.randint(0, 2, n)  # 0 = Z basis, 1 = X basis
    return bits, bases

def prepare_qubits(bits, bases):
    """Create quantum states based on Alice's bits and bases"""
    qc_list = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if basis == 1:
            qc.h(0)  # Apply Hadamard if in X basis
        if bit == 1:
            qc.x(0)  # Apply X gate if bit is 1
        qc_list.append(qc)
    return qc_list

def measure_qubits(qc_list, bases):
    """Bob measures the qubits"""
    results = []
    backend = Aer.get_backend('aer_simulator')
    for qc, basis in zip(qc_list, bases):
        if basis == 1:
            qc.h(0)  # Apply Hadamard back if measuring in X basis
        qc.measure(0, 0)
        result = backend.run(qc, shots=10).result()
        measured_bit = int(list(result.get_counts().keys())[0])  # Extract bit
        results.append(measured_bit)
    return results

# Simulate BB84
n = 100
alice_bits, alice_bases = generate_bb84_bits(n)
qc_list = prepare_qubits(alice_bits, alice_bases)
bob_bases = np.random.randint(0, 2, n)
bob_measurements = measure_qubits(qc_list, bob_bases)

## 2. AI-Assisted Attack Detection in Noisy Quantum Channels
### Step 2: Simulating a Quantum Channel with Noise
We simulate an intercept-resend attack or decoherence noise using depolarizing channels.

In [7]:
from qiskit_aer.noise import NoiseModel, errors



In [9]:
def create_noise_model():
    """Define a depolarizing noise model to simulate an eavesdropper"""
    noise_model = NoiseModel()
    error = errors.depolarizing_error(0.05, 1)  # 5% depolarization probability
    noise_model.add_all_qubit_quantum_error(error, ['measure'])
    return noise_model

def simulate_noisy_bb84(qc_list, bases):
    """Simulate Bob's measurement with a noisy quantum channel"""
    results = []
    backend = Aer.get_backend('aer_simulator')
    noise_model = create_noise_model()
    for qc, basis in zip(qc_list, bases):
        if basis == 1:
            qc.h(0)
        qc.measure(0, 0)
        result = backend.run(qc, noise_model=noise_model, shots=10).result()
        measured_bit = int(list(result.get_counts().keys())[0])
        results.append(measured_bit)
    return results

bob_noisy_measurements = simulate_noisy_bb84(qc_list, bob_bases)


## 3. Using Machine Learning for Attack Detection
An AI model can analyze deviations in the quantum channel noise to detect potential eavesdropping.

### Step 3: Training an ML Model for Attack Detection
We will train a Neural Network (NN) to differentiate between normal and attack scenarios based on the quantum bit error rate (QBER).

Feature Engineering
Input: Alice’s and Bob’s measurements, bit mismatch rates.
Output: Probability of an eavesdropping attack.
Data Preparation

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Generate dataset
def generate_qkd_dataset(samples=1000):
    data = []
    for _ in range(samples):
        # Simulate normal transmission
        alice_bits, alice_bases = generate_bb84_bits(100)
        qc_list = prepare_qubits(alice_bits, alice_bases)
        bob_bases = np.random.randint(0, 2, 100)
        bob_measurements = measure_qubits(qc_list, bob_bases)

        # Simulate attack/noise
        if np.random.rand() < 0.5:  # 50% chance of attack
            bob_measurements = simulate_noisy_bb84(qc_list, bob_bases)
            label = 1  # Attack
        else:
            label = 0  # No Attack

        qber = np.sum(np.array(alice_bits) != np.array(bob_measurements)) / 100
        data.append([qber, label])

    return pd.DataFrame(data, columns=['QBER', 'Attack'])

df = generate_qkd_dataset()
X_train, X_test, y_train, y_test = train_test_split(df[['QBER']], df['Attack'], test_size=0.2)

# Train an AI Model (Random Forest Classifier)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Test AI Model
print("Model Accuracy:", clf.score(X_test, y_test))


## 4. Reinforcement Learning for Dynamic Key Rate Allocation
Instead of fixed QKD key rates, we can use reinforcement learning (RL) to optimize transmission rates based on channel conditions.

Approach:

Define the state as channel conditions and noise levels.
Define actions as key rate adjustments.
Use Q-learning to maximize secure key generation.
### Step 4: RL-Based Key Rate Allocation

In [ ]:
import gym
import numpy as np

class QKDSimulator(gym.Env):
    def __init__(self):
        self.state = 0.5  # QBER as state
        self.action_space = np.array([0.1, 0.2, 0.3])  # Adjust key rate
        self.reward = 0

    def step(self, action):
        self.state += np.random.normal(0, 0.01)  # Simulate noise variation
        self.reward = -abs(self.state - action)  # Reward based on key rate efficiency
        return self.state, self.reward

    def reset(self):
        self.state = 0.5
        return self.state

env = QKDSimulator()
for _ in range(10):
    action = np.random.choice(env.action_space)
    state, reward = env.step(action)
    print(f"Action: {action}, New QBER: {state:.3f}, Reward: {reward:.3f}")


## Conclusion & Next Steps
This project integrates AI into QKD for attack detection, error correction, and dynamic key rate optimization.
Possible improvements:
* Implement deep learning models for attack detection.
* Test on real quantum hardware using IBMQ.
* Extend reinforcement learning for network-wide QKD optimization.
